# TEAM benchmark 7

https://www.compumag.org/wp/wp-content/uploads/2018/06/problem7.pdf

Solve
\begin{align*}
\nabla\times(\nu\nabla\times\mathbf{A}) + j\omega\sigma \mathbf{A} = \mathbf{J}
\end{align*}
with
\begin{align*}
\mathbf{A}\times\mathbf{n} = \mathbf{0}
\end{align*}
on the far boundary. The magnetic permability $\mu_0 =\nu_0^{-1} = 4\pi \cdot 10^{-7}$ H/m  is set to the permability of vacuum and the electric conductivity $\sigma_{Al} = 3.526\cdot10^7$ S/m in the aluminum plate is used.


In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
from netgen.meshing import MeshingParameters

## Generate Geometry

The aluminum plate:

In [ ]:
wpplate = WorkPlane(Axes((0,0,0),Z,X))
f = wpplate.Rectangle(0.294,0.294).MoveTo(0.018,0.018).Rectangle(0.108,0.108).Reverse().Face()
plate = f.Extrude(0.019)

plate.faces.maxh=0.02
plate.solids.name="plate"
plate.faces.col=(0.3,0.3,0.3,1)

The coil:

In [ ]:
wp = WorkPlane(Axes(p=(0.294,0.000,0.049), n=Z, h=Y)) # origin in 

coil2di = wp.MoveTo(0.100,0.100).RectangleC(0.100,0.100).Offset(0.025).Face()
coil2do = wp.MoveTo(0.100,0.100).RectangleC(0.100,0.100).Offset(0.050).Face()
coil2d = coil2do-coil2di

cutout = wp.MoveTo(0.100,0.100).RectangleC(0.300,0.100).Face() + \
         wp.MoveTo(0.100,0.100).RectangleC(0.100,0.300).Face()

coil2d = Glue( [coil2d*cutout, coil2d-cutout])

Draw (coil2d)
coil = coil2d.Extrude(0.100)
coil.solids.name="coil"
Draw (coil);

In [ ]:
bounding_box = Box(Pnt(-.2, -.2, -.2), Pnt(0.5, 0.5, 0.5))
bounding_box.faces.Max(X).name="right"
bounding_box.faces.Min(X).name="left"
bounding_box.faces.Max(Y).name="back"
bounding_box.faces.Min(Y).name="front"
bounding_box.faces.Max(Z).name="top"
bounding_box.faces.Min(Z).name="bottom"
bounding_box.solids.name="air"

bounding_box.faces.col = (0,0,1,0.3)

bounding_box = bounding_box-coil-plate
shape = Glue([ coil, plate, bounding_box])
Draw (shape);

In [ ]:
geo = OCCGeometry(shape)

Restrict around measerung points

In [ ]:
xi_msm = np.array([0, 18, 36, 54, 72, 90, 108, 126, 144, 162, 180, 198, 216, 234, 252, 270, 288]) * 1e-3

mp = MeshingParameters ( maxh =1)
# for A1B1 measurement
[mp.RestrictH(x, 0.072, 0.034, h= 0.01/3) for x in xi_msm];
# for A2B2 measurement
[mp.RestrictH(x, 0.144, 0.034, h= 0.01/3) for x in xi_msm];
# for A3B3 measurement
[mp.RestrictH(x, 0.072, 0.019, h= 0.01/3) for x in xi_msm];
# for A4B4 measurement
[mp.RestrictH(x, 0.072, 0.000, h= 0.01/3) for x in xi_msm];


In [ ]:
mesh = Mesh(geo.GenerateMesh(maxh=0.1, mp=mp))
mesh.Curve(3)
print("mats", set(mesh.GetMaterials()))
print("number of elements:", mesh.ne)
clipping_settings={"Clipping":{"enable":True, "z":-1, "dist":0}}

Draw(mesh, settings = clipping_settings);

## Define Current in Coil

In [ ]:
coilrect_xmin, coilrect_xmax = 0.294 - 0.150, 0.294-0.050
coilrect_ymin, coilrect_ymax = 0.050, 0.150

def Project(val, minval, maxval):
    return IfPos(val-minval, IfPos(val-maxval, maxval, val), minval)

projx = Project(x, coilrect_xmin, coilrect_xmax)
projy = Project(y, coilrect_ymin, coilrect_ymax)
Jtau = CF( (projy-y, x-projx, 0) )
Jtau /= Norm(Jtau)
J_coil = mesh.MaterialCF({"coil.*":Jtau}, default=CF((0, 0, 0)))

In [ ]:
clipping_settings={"Clipping":{"enable":True, "z":-1, "dist":-0.05}, "Objects":{"Clipping Plane":False, "Vectors":True}, "Vectors":{"grid_size":100}}
clipping_settings.update({"camera":{"transformations":[{"type":"move", "dir":(0,0,1), "dist":1.9}]}})
Draw(J_coil, mesh,settings=clipping_settings, draw_surf=False); # top right, mesh, settings=clipping_settings)

## FEM

In [ ]:
mu = 4*pi*1e-7
sigma = mesh.MaterialCF({"plate":35.26e6}, default=1)

In [ ]:
J0 = 2742 / (0.025 * 0.100)
print(f"J0 = {J0} A/m**2")

def Solve(f):
    omega = 2*pi*f
    fes = HCurl(mesh, order=3, dirichlet="top|back|right|bottom|front|left", complex=True)
    # fes = HCurl(mesh, order=3, complex=True)
    print("free dofs", sum(fes.FreeDofs()))
    u, v = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += 1/mu*curl(u)*curl(v)*dx
    a += 1j*omega*sigma * u*v*dx
    
    f = LinearForm(-J0*J_coil*v * dx("coil.*"))
    
    A = GridFunction(fes)
    pre = Preconditioner(a, type="bddc", inverse="sparsecholesky")
    with TaskManager():
        a.Assemble()
        f.Assemble()
        inv = solvers.CGSolver(mat=a.mat, pre=pre, printrates='\r')
        A.vec[:] = inv*f.vec

    B = curl(A)
    J = -1j*omega*sigma * A
    return {"A":A, "B":B, "J":J}

with TaskManager():
    ret = Solve(f=50)
B, J = ret["B"], ret["J"]

## Draw the B and J field

In [ ]:
clipping_settings={"Clipping":{"enable":True, "y":1, "z":0, "dist":0.1}, "Objects":{"Clipping Plane":False, "Vectors":True}, "Vectors":{"grid_size":100}}
clipping_settings.update({"camera":{"transformations":[{"type":"rotateX", "angle":-90}, {"type":"move", "dir":(0,0,1.5), "dist":1}]}})
Draw(B.real, mesh, settings=clipping_settings, max = 10e-3, min = 0, draw_surf=False);

In [ ]:
clipping_settings={"Clipping":{"enable":True, "y":0, "z":-1, "dist":-0.132}, "Objects":{"Clipping Plane":False, "Vectors":True}, "Vectors":{"grid_size":150}}

clipping_settings.update({"camera":{"transformations":[{"type":"move", "dir":(0,0,1), "dist":2.2}]}})
Draw(mesh.MaterialCF({"plate":J.imag}, default = CF((0, 0, 0))), mesh, settings = clipping_settings, min = 0, max = 8e5);

## Evaluation of B and J according to Description

In [ ]:
Bz_A1B1_50Hz_ref = np.array([ -4.9  -1.16j, -17.88 +2.48j, -22.13 +4.15j, -20.19 +4.j,   -15.67 +3.07j,
   0.36 +2.31j,  43.64 +1.89j,  78.11 +4.97j,  71.55+12.61j , 60.44+14.15j,
  53.91+13.04j,  52.62+12.4j,  53.81+12.05j, 56.91+12.27j,  59.24+12.66j,
  52.78 +9.96j,  27.61 +2.26j])
               
Bz_A2B2_50Hz_ref = np.array([-1.83-1.63j, -8.5-0.6j, -13.6-0.43j, -15.21+0.11j, -14.48+1.26j, -5.62+3.4j,
 28.77+6.53j, 60.34+10.25j, 61.84+11.83j, 56.64+11.83j, 53.4+11.01j, 52.36+10.58j, 53.93+10.8j, 56.82+10.54j, 
 59.48+10.62j, 52.08+9.03j, 26.56+1.79j])

Jy_A3B3_50Hz_ref = np.array([0.249-0.629j,  0.685-0.873j,  0+0j,  0+0j,  0+0j,  0+0j,  0+0j,  -0.015-0.593j,
             -0.103-0.249j,  -0.061-0.101j,  -0.004-0.001j,  0.051+0.087j,  0.095+0.182j,  0.135+0.322j,  
             0.104+0.555j,  -0.321+0.822j,  -0.687+0.855j,])


Jy_A4B4_50Hz_ref = np.array([0.461-0.662j,  0.621-0.664j,  0+0j,  0+0j,  0+0j,  0+0j,  0+0j,  1.573-1.027j,
             0.556-0.757j,  0.237-0.364j,  0.097-0.149j,  -0.034+0.015j,  -0.157+0.154j,  -0.305+0.311j,
             -0.478+0.508j,  -0.66+0.747j,  -1.217+1.034j])


### Bz on A1 to B1

In [ ]:
Bz_A1B1 = np.array([B[2](mesh(x, 0.072, 0.034)) for x in xi_msm])

plt.figure(1)
plt.clf()
plt.title("Evaluate B.z on Line A1 - B1")
plt.plot(xi_msm, -1e4 * Bz_A1B1.real, "b-x", label="sim Bz.real")
plt.plot(xi_msm, 1e4 * Bz_A1B1.imag, "r-x", label="sim Bz.imag")
plt.plot(xi_msm, Bz_A1B1_50Hz_ref.real, "b--x", label="ref Bz.real")
plt.plot(xi_msm, Bz_A1B1_50Hz_ref.imag, "r--x", label="ref Bz.imag")
plt.ylabel("B in Gauss = $10^{-4}$ T")
plt.legend()
plt.show()

### Bz on A2 to B2

In [ ]:
Bz_A2B2 = np.array([B[2](mesh(x, 0.144, 0.034)) for x in xi_msm])

plt.figure(2)
plt.clf()
plt.title("Evaluate B.z on Line A2 - B2")
plt.plot(xi_msm, -1e4 * Bz_A2B2.real, "b-x", label="sim Bz.real")
plt.plot(xi_msm, 1e4 * Bz_A2B2.imag, "r-x", label="sim Bz.imag")
plt.plot(xi_msm, Bz_A2B2_50Hz_ref.real, "b--x", label="ref Bz.real")
plt.plot(xi_msm, Bz_A2B2_50Hz_ref.imag, "r--x", label="ref Bz.imag")
plt.ylabel("B in Gauss = $10^{-4}$ T")
plt.legend()
plt.show()

In [ ]:
clipping = { "function" : True,  "pnt" : (0,0.072,0), "vec" : (0,1,0) }

Draw (J[1].real, mesh, clipping = clipping, min=-0.5e6, max=0.5e6 )
Draw (J[1].imag, mesh, clipping = clipping, min=-0.5e6, max=0.5e6 )

## Jy on A3 to B3 - plate top:

In [ ]:
Jy = GridFunction(H1(mesh, order=1, complex=True))
Jy.Set (J[1], definedon=mesh.Materials("plate"))
xi_sim = np.linspace(0,0.288,500)

In [ ]:
# Jy_A3B3 = np.array([J[1](mesh(x, 0.072, 0.019-1e-5)) for x in xi_msm])
Jy_A3B3_sim = np.array([Jy(mesh(x, 0.072, 0.019)) for x in xi_sim])

plt.figure(3)
plt.clf()
plt.title("Evaluate J.y on Line A3 - B3")
plt.plot( xi_sim, 1e-6 *Jy_A3B3_sim.real, "b", label="sim Jy.real")
plt.plot( xi_sim, 1e-6 * Jy_A3B3_sim.imag, "r", label="sim Jy.imag")
plt.plot(xi_msm, Jy_A3B3_50Hz_ref.real, "b--x", label="ref Jy.real")
plt.plot(xi_msm, Jy_A3B3_50Hz_ref.imag, "r--x", label="ref Jy.imag")
plt.ylabel("J in $10^{6}$ A/mm$^2$")
plt.ylim([min(list(Jy_A3B3_50Hz_ref.real)+list(Jy_A3B3_50Hz_ref.imag)), max(list(Jy_A3B3_50Hz_ref.real)+list(Jy_A3B3_50Hz_ref.imag))])
plt.legend()
plt.show()

## Jy on A4 to B4  - plate bottom

In [ ]:
# Jy_A4B4 = np.array([J[1](mesh(x, 0.072, 0.0+1e-3)) for x in xi_msm])
Jy_A4B4_sim = np.array([Jy(mesh(x, 0.072, 0.0+1e-3)) for x in xi_sim])

plt.figure(4)
plt.clf()
plt.title("Evaluate J.y on Line A4 - B4")
plt.plot( xi_sim, - 1e-6 *Jy_A4B4_sim.imag, "b", label="sim Jy.real")
plt.plot( xi_sim, 1e-6 * Jy_A4B4_sim.real, "r", label="sim Jy.imag")
plt.plot(xi_msm, Jy_A4B4_50Hz_ref.real, "b--x", label="ref Jy.real")
plt.plot(xi_msm, Jy_A4B4_50Hz_ref.imag, "r--x", label="ref Jy.imag")
plt.ylabel("J in $10^{6}$ A/mm$^2$")
plt.ylim([min(list(Jy_A4B4_50Hz_ref.real)+list(Jy_A4B4_50Hz_ref.imag)), max(list(Jy_A4B4_50Hz_ref.real)+list(Jy_A4B4_50Hz_ref.imag))])
plt.legend()
plt.show()